In [1]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import lightning.pytorch as pl
from omegaconf import OmegaConf
from pytorch_lightning import Trainer


# Task Description
Automatic Speech Recognition (ASR) systems typically generate text with no punctuation and capitalization of the words. 
This tutorial explains how to implement a model in NeMo that will predict punctuation and capitalization for each word in a sentence to make ASR output more readable and to boost performance of the named entity recognition, machine translation or text-to-speech models. 
We'll show how to train a model for this task using a pre-trained BERT model. 
For every word in our training dataset we’re going to predict:

- punctuation mark that should follow the word and
- whether the word should be capitalized

# Dataset
This model can work with any dataset as long as it follows the format specified below. 
The training and evaluation data is divided into *2 files: text.txt and labels.txt*. 
Each line of the **text.txt** file contains text sequences, where words are separated with spaces: [WORD] [SPACE] [WORD] [SPACE] [WORD], for example:



```
when is the next flight to new york
the next flight is ...
...
```



The **labels.txt** file contains corresponding labels for each word in text.txt, the labels are separated with spaces. Each label in labels.txt file consists of 2 symbols:

- the first symbol of the label indicates what punctuation mark should follow the word (where O means no punctuation needed);
- the second symbol determines if a word needs to be capitalized or not (where U indicates that the word should be upper cased, and O - no capitalization needed.)

In this tutorial, we are considering only commas, periods, and question marks the rest punctuation marks were removed. To use more punctuation marks, update the dataset to include desired labels, no changes to the model needed. 

Each line of the **labels.txt** should follow the format: 
[LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (for labels.txt). 
For example, labels for the above text.txt file should be:



```
OU OO OO OO OO OO OU ?U
OU OO OO OO ...
...
```



The complete list of all possible labels for this task used in this tutorial is: `OO, ,O, .O, ?O, OU, ,U, .U, ?U.`

## Download and preprocess the data¶

In this notebook we are going to use a subset of English examples from the [Tatoeba collection of sentences](https://tatoeba.org/eng) this script will download and preprocess the Tatoeba data [NeMo/examples/nlp/token_classification/get_tatoeba_data.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/get_tatoeba_data.py). Note, for further experiments with the model, set NUM_SAMPLES=-1 and consider including other datasets to improve model performance. 


In [2]:
DATA_DIR = '/workspace/datasets/punc_restore'
WORK_DIR = '/workspace/NeMo-W/examples/nlp/token_classification'
MODEL_CONFIG = "punctuation_capitalization_config.yaml"

# model parameters
TOKENS_IN_BATCH = 1024
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002
NUM_SAMPLES = 10000

In [3]:
## download get_tatoeba_data.py script to download and preprocess the Tatoeba data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(f"{WORK_DIR}/get_tatoeba_data.py"):
    print('Downloading get_tatoeba_data.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/get_tatoeba_data.py', WORK_DIR)
else:
    print ('get_tatoeba_data.py already exists')

NameError: name 'BRANCH' is not defined

In [ ]:
# download and preprocess the data
# --clean_dir flag deletes raw Tataoeba data, remove the flag to avoid multiple data downloads if you want to experiment with the dataset size
! python $WORK_DIR/get_tatoeba_data.py --data_dir $DATA_DIR --num_sample $NUM_SAMPLES --clean_dir

after execution of the above cell, your data folder will contain the following 4 files needed for training (raw Tatoeba data could be present if `--clean_dir` was not used):
- labels_dev.txt
- labels_train.txt
- text_dev.txt
- text_train.txt


In [3]:
! ls -l $DATA_DIR

total 286444
-rw-rw-r-- 1 1002 1003  4970881 Dec  8 02:06 cached.__text_dev.txt__labels_dev.txt__.BertTokenizer.max_seq_length512.vocab30522.num_samples10000.punctuation_capitalization.pkl
-rw-rw-r-- 1 1002 1003  5399489 Dec  8 02:06 cached.__text_train.txt__labels_train.txt__.BertTokenizer.max_seq_length512.vocab30522.num_samples10000.punctuation_capitalization.pkl
-rw-rw-r-- 1 1002 1003 61557003 Dec  8 02:07 clean_rus_sentences.txt
-rw-rw-r-- 1 1002 1003 12416096 Dec  8 02:06 dev.txt
drwxrwxr-x 2 1002 1003     4096 Dec  8 02:05 label_id_files_for_nemo_checkpoint
-rw-rw-r-- 1 1002 1003  3375729 Dec  8 02:06 labels_dev.txt
-rw-rw-r-- 1 1002 1003 13721988 Dec  8 02:06 labels_train.txt
drwxrwxr-x 2 1002 1003     4096 Dec  8 02:05 processed
-rw-rw-r-- 1 1002 1003 80739921 Dec  8 02:07 sentences.tsv
-rw-rw-r-- 1 1002 1003 11920999 Dec  8 02:06 text_dev.txt
-rw-rw-r-- 1 1002 1003 48184291 Dec  8 02:07 text_train.txt
-rw-rw-r-- 1 1002 1003 50171907 Dec  8 02:07 train.txt
drwxrwxr-x 2 1002 10

In [4]:
# let's take a look at the data 
print('Text:')
! head -n 5 $DATA_DIR/text_train.txt

print('\nLabels:')
! head -n 5 $DATA_DIR/labels_train.txt

Text:
один раз в жизни я делаю хорошее дело и оно бесполезно
мне пора идти спать
что ты делаешь
что это
у меня нет слов



Labels:
OU OO OO OO OO OO OO .O OU OO .O
OU OO OO .O
OU OO ?O
OU ?O
OU OO OO .O


As you see, `get_tatoeba_data.py` script provides not only downloads Tatoeba but also creates labels. If you wish to preprocess your own data, use [examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py) script:

```
NEMO_ROOT = "<Path_to_NeMo>"
!python $NEMO_ROOT/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py \
    --source_file $DATA_DIR/text_train.txt \
    --output_dir $DATA_DIR/my_train_preprocessed
```

In [ ]:
!ls $DATA_DIR/my_train_preprocessed -l

# Tarred dataset

If your dataset is too large to be stored in memory, you can use tarred dataset. A tarred dataset is a collection of tarred files which contain batches ready for passing into a model.

All tar files will contain identical number of batches, so if number of batches in the dataset is not evenly divisible by parameter `--num_batches_per_tar_file` value, then up to `--num_batches_per_tar_file - 1` batches may be lost.

In [5]:
# Number of lines in text and labels files
!wc -l $DATA_DIR/text_train.txt
!wc -l $DATA_DIR/labels_train.txt

824802 /workspace/datasets/punc_restore/text_train.txt


824802 /workspace/datasets/punc_restore/labels_train.txt


In [ ]:
## download script to prepare tarred dataset
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(f"{WORK_DIR}/create_punctuation_capitalization_tarred_dataset.py"):
    print('Downloading create_punctuation_capitalization_tarred_dataset.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/create_punctuation_capitalization_tarred_dataset.py', WORK_DIR)
else:
    print ("create_punctuation_capitalization_tarred_dataset.py script already exists")

!python $WORK_DIR/create_punctuation_capitalization_tarred_dataset.py \
    --text $DATA_DIR/text_train.txt \
    --labels $DATA_DIR/labels_train.txt \
    --output_dir $DATA_DIR/train_tarred \
    --num_batches_per_tarfile 5 \
    --tokens_in_batch 1024 \
    --lines_per_dataset_fragment 4000 \
    --tokenizer_name bert-base-uncased \
    --n_jobs 2

In [ ]:
!ls $DATA_DIR/train_tarred -l

In [ ]:
!ls $DATA_DIR/train_tarred/*.tar | wc -l  # number of tar files

In [ ]:
!ls $DATA_DIR/train_tarred/ | grep -v '.tar'  # all not tar files

If you wish to use tarred dataset, then you need to 
- pass metadata JSON file in config parameter `model.train_ds.tar_metadata_file`,
- set `model.train_ds.use_tarred_dataset=true`.

# Model Configuration

In the Punctuation and Capitalization Model, we are jointly training two token-level classifiers on top of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model: 
- one classifier to predict punctuation and
- the other one - capitalization.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, dataset and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

See [docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html#training-punctuation-and-capitalization-model) for full config description.

In [3]:
# download the model's configuration file 
config_dir = WORK_DIR + '/conf/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

config file already exists


In [6]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/conf/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

/workspace/NeMo-W/examples/nlp/token_classification/conf/punctuation_capitalization_config.yaml
pretrained_model: null
trainer:
  devices: 1
  num_nodes: 1
  max_epochs: 3
  max_steps: -1
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  precision: 16
  accelerator: gpu
  strategy: ddp
  enable_checkpointing: false
  logger: false
  log_every_n_steps: 1
  val_check_interval: 1.0
exp_manager:
  exp_dir: null
  name: Punctuation_and_Capitalization
  create_tensorboard_logger: true
  create_checkpoint_callback: true
model:
  class_labels:
    punct_labels_file: punct_label_ids.csv
    capit_labels_file: capit_label_ids.csv
  common_dataset_parameters:
    pad_label: O
    ignore_extra_tokens: false
    ignore_start_end: true
    punct_label_ids: null
    capit_label_ids: null
    label_vocab_dir: null
  train_ds:
    use_tarred_dataset: false
    ds_item: ???
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    tokens_in_batch: 15

# Setting up Data within the config

Among other things, the config file contains dictionaries called `common_dataset_parameters`, `train_ds` and `validation_ds`. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

Specify paths directories with train and dev datasets in parameters `train_ds.ds_item` and `validation_ds.ds_item`.

I you want to use multiple datasets for evaluation, specify paths to the directory(ies) with evaluation file(s) in the following way:

`model.validation_ds.ds_item=[PATH_TO_DEV1,PATH_TO_DEV2]` (Note no space between the paths and square brackets).

Also notice that some configs, including `model.train_ds.ds_item`, have `???` in place of values, this values are required to be specified by the user.

Let's now add the data directory path to the config.

In [7]:
# in this tutorial train and dev data is located in the same folder
config.model.train_ds.ds_item = DATA_DIR
config.model.validation_ds.ds_item=DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let's first instantiate a Trainer object!

In [8]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

devices: 1
num_nodes: 1
max_epochs: 3
max_steps: -1
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 16
accelerator: gpu
strategy: ddp
enable_checkpointing: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0



In [20]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

# Reduces maximum number of epochs to 1 for a quick training
config.trainer.max_epochs = 15

# Remove distributed training flags
config.trainer.strategy = 'auto'

trainer = Trainer(**config.trainer)

[NeMo W 2024-12-08 02:27:18 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
    
INFO: Using 16bit Automatic Mixed Precision (AMP)
I1208 02:27:18.933322 128621095987008 rank_zero.py:63] Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
I1208 02:27:18.959913 128621095987008 rank_zero.py:63] GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
I1208 02:27:18.961236 128621095987008 rank_zero.py:63] TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
I1208 02:27:18.962811 128621095987008 rank_zero.py:63] IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
I1208 02:27:18.963886 128621095987008 rank_zero.py:63] HPU available: False, using: 0 HPUs
INFO: `Trainer(val_check_interval=1.0)` was confi

# Setting up a NeMo Experiment¶

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [21]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

[NeMo I 2024-12-08 02:28:05 exp_manager:396] Experiments will be logged at /workspace/NeMo-W/tutorials/nlp/nemo_experiments/Punctuation_and_Capitalization/2024-12-08_02-24-38
[NeMo I 2024-12-08 02:28:05 exp_manager:856] TensorboardLogger has been set up


'/workspace/NeMo-W/tutorials/nlp/nemo_experiments/Punctuation_and_Capitalization/2024-12-08_02-24-38'

# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model.

In [22]:
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())

PRETRAINED_BERT_MODEL = "bert-base-uncased"

['google-bert/bert-base-uncased', 'google-bert/bert-large-uncased', 'google-bert/bert-base-cased', 'google-bert/bert-large-cased', 'google-bert/bert-base-multilingual-uncased', 'google-bert/bert-base-multilingual-cased', 'google-bert/bert-base-chinese', 'google-bert/bert-base-german-cased', 'google-bert/bert-large-uncased-whole-word-masking', 'google-bert/bert-large-cased-whole-word-masking', 'google-bert/bert-large-uncased-whole-word-masking-finetuned-squad', 'google-bert/bert-large-cased-whole-word-masking-finetuned-squad', 'google-bert/bert-base-cased-finetuned-mrpc', 'google-bert/bert-base-german-dbmdz-cased', 'google-bert/bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased', 'distilbert-base-uncased', 'distilbert-bas

In [23]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.train_ds.num_samples = NUM_SAMPLES
config.model.validation_ds.num_samples = NUM_SAMPLES


Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [24]:
# initialize the model
# during this stage, the dataset and data loaders we'll be prepared for training and evaluation
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)

[NeMo I 2024-12-08 02:28:13 tokenizer_utils:128] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /root/.cache/huggingface/nemo_nlp_tmp/ae0d012864bdb2474ba67537c3f5e0fa/vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


[NeMo W 2024-12-08 02:28:13 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
      warnings.warn(
    
[NeMo W 2024-12-08 02:28:13 modelPT:272] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2024-12-08 02:28:13 punctuation_capitalization_dataset:1192] Features restored from /workspace/datasets/punc_restore/cached.__text_train.txt__labels_train.txt__.BertTokenizer.max_seq_length512.vocab30522.num_samples10000.punctuation_capitalization.pkl


Batch mark up:   0%|          | 0/10000 [00:00<?, ?query/s][NeMo W 2024-12-08 02:28:13 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too small. Current length of sequences in batch is 144. Batch size will be reduced to 7. tokens_in_batch=1024. The batch includes sequences from 9984 to 9990.
[NeMo W 2024-12-08 02:28:13 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too small. Current length of sequences in batch is 184. Batch size will be reduced to 5. tokens_in_batch=1024. The batch includes sequences from 9991 to 9995.
[NeMo W 2024-12-08 02:28:13 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too smal

[NeMo I 2024-12-08 02:28:13 punctuation_capitalization_dataset:1192] Features restored from /workspace/datasets/punc_restore/cached.__text_dev.txt__labels_dev.txt__.BertTokenizer.max_seq_length512.vocab30522.num_samples10000.punctuation_capitalization.pkl


Batch mark up:   0%|          | 0/10000 [00:00<?, ?query/s][NeMo W 2024-12-08 02:28:13 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too small. Current length of sequences in batch is 136. Batch size will be reduced to 7. tokens_in_batch=1024. The batch includes sequences from 9968 to 9974.
[NeMo W 2024-12-08 02:28:13 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too small. Current length of sequences in batch is 160. Batch size will be reduced to 6. tokens_in_batch=1024. The batch includes sequences from 9975 to 9980.
[NeMo W 2024-12-08 02:28:13 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too smal

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [25]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [26]:
# start the training
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-12-08 02:28:18 modelPT:770] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 2e-05
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2024-12-08 02:28:18 lr_scheduler:923] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x74f944821d20>" 
    will be used during training (effective maximum steps = 5850) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 5850
    )



  | Name             | Type             | Params
------------------------------------------------------
0 | metrics          | ModuleDict       | 0     
1 | bert_model       | BertEncoder      | 109 M 
2 | punct_classifier | TokenClassifier  | 3.1 K 
3 | capit_classifier | TokenClassifier  | 1.5 K 
4 | loss             | CrossEntropyLoss | 0     
5 | agg_loss         | AggregatorLoss   | 0     
------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.947   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2024-12-08 02:28:19 punctuation_capitalization_model:342] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                        100.00       3.72       7.17        269
    , (label_id: 1)                                          5.04      68.42       9.39         19
    . (label_id: 2)                                          0.00       0.00       0.00         55
    ? (label_id: 3)                                          1.19       9.09       2.11         11
    -------------------
    micro avg                                                6.78       6.78       6.78        354
    macro avg                                               26.56      20.31       4.67        354
    weighted avg                                            76.30       6.78       6.02        354
    
[NeMo I 2024-12-08 02:28:19 punctuation_capitalization_model:343] Capitalization rep

...

Validation: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2024-12-08 02:40:50 punctuation_capitalization_model:342] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         97.20      98.83      98.01      43398
    , (label_id: 1)                                         78.80      67.63      72.79       3281
    . (label_id: 2)                                         92.23      93.35      92.79       8461
    ? (label_id: 3)                                         77.20      61.80      68.64       1835
    -------------------
    micro avg                                               95.03      95.03      95.03      56975
    macro avg                                               86.36      80.40      83.06      56975
    weighted avg                                            94.76      95.03      94.84      56975
    
[NeMo I 2024-12-08 02:40:50 punctuation_capitalization_model:343] Capitalization rep

INFO: Epoch 14, global step 5850: 'val_loss' was not in top 3
I1208 02:40:50.360040 128621095987008 rank_zero.py:63] Epoch 14, global step 5850: 'val_loss' was not in top 3


[NeMo I 2024-12-08 02:40:53 punctuation_capitalization_dataset:1821] Shuffling training dataset


[NeMo W 2024-12-08 02:40:53 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too small. Current length of sequences in batch is 144. Batch size will be reduced to 7. tokens_in_batch=1024. The batch includes sequences from 9984 to 9990.
[NeMo W 2024-12-08 02:40:53 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too small. Current length of sequences in batch is 184. Batch size will be reduced to 5. tokens_in_batch=1024. The batch includes sequences from 9991 to 9995.
[NeMo W 2024-12-08 02:40:53 punctuation_capitalization_dataset:1572] Could not create batch with multiple of 8 size. Probably, there is a too long sequence in the dataset or parameter `tokens_in_batch` is too small. Current length of sequences in batch is 280. Batch size 

# Training using tarred dataset

In [ ]:
config = OmegaConf.load(config_path)
config.model.train_ds.ds_item = f'{DATA_DIR}/train_tarred'
config.model.train_ds.use_tarred_dataset = True
# Only metadata file name is required if `use_tarred_dataset=true`.
config.model.train_ds.tar_metadata_file = 'metadata.punctuation_capitalization.tokens1024.max_seq_length512.bert-base-uncased.json'
config.model.validation_ds.ds_item = DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Trainer
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.max_epochs = 1
config.trainer.strategy = 'auto'

# Exp manager
config.exp_manager.explicit_log_dir = 'tarred_experiment'

config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.validation_ds.num_samples = NUM_SAMPLES

In [ ]:
trainer = pl.Trainer(**config.trainer)
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)
trainer.fit(model)

# Inference using a pretrained model

To see how the model performs, let’s run inference on a few examples.

In [ ]:
print(f"Available_models: {nemo_nlp.models.PunctuationCapitalizationModel.get_available_model_names()}\n")

pretrained_model = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained("punctuation_en_distilbert")
# define the list of queiries for inference
queries = [
        'we bought four shirts and one mug from the nvidia gear store in santa clara',
        'what can i do for you today',
        'how are you',
        'how is the weather in',
        'один раз в жизни я делаю хорошее дело и оно бесполезно',
    ]
inference_results = pretrained_model.add_punctuation_capitalization(queries)
print()

for query, result in zip(queries, inference_results):
    print(f'Query   : {query}')
    print(f'Combined: {result.strip()}\n')

Available_models: ['punctuation_en_bert', 'punctuation_en_distilbert']

[NeMo I 2024-12-08 02:41:27 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/punctuation_en_distilbert/versions/1.0.0rc1/files/punctuation_en_distilbert.nemo to /root/.cache/torch/NeMo/NeMo_2.0.0rc0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo
[NeMo I 2024-12-08 02:41:34 common:815] Instantiating model from pre-trained checkpoint
[NeMo I 2024-12-08 02:41:39 tokenizer_utils:128] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmp83j9_ug8/tokenizer.vocab_file, merges_files: None, special_tokens_dict: {}, and use_fast: False


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

[NeMo W 2024-12-08 02:41:45 save_restore_connector:408] src path does not exist or it is not a path in nemo file. src value I got was: punct_label_ids.csv. Absolute: /workspace/NeMo-W/tutorials/nlp/punct_label_ids.csv
[NeMo W 2024-12-08 02:41:45 punctuation_capitalization_model:719] The artifact `class_labels.punct_labels_file` was not found in checkpoint. Will rely on `punct_label_ids` parameter
[NeMo W 2024-12-08 02:41:45 save_restore_connector:408] src path does not exist or it is not a path in nemo file. src value I got was: capit_label_ids.csv. Absolute: /workspace/NeMo-W/tutorials/nlp/capit_label_ids.csv
[NeMo W 2024-12-08 02:41:45 punctuation_capitalization_model:741] The artifact `class_labels.capit_labels_file` was not found in checkpoint. Will rely on `capit_label_ids` parameter


[NeMo I 2024-12-08 02:41:45 save_restore_connector:263] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.0.0rc0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo.
[NeMo I 2024-12-08 02:41:45 punctuation_capitalization_model:1167] Using batch size 4 for inference
[NeMo I 2024-12-08 02:41:45 punctuation_capitalization_infer_dataset:127] Max length: 19
[NeMo I 2024-12-08 02:41:45 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2024-12-08 02:41:45 data_preprocessing:406] Min: 3 |                  Max: 17 |                  Mean: 8.0 |                  Median: 6.0
[NeMo I 2024-12-08 02:41:45 data_preprocessing:412] 75 percentile: 9.50
[NeMo I 2024-12-08 02:41:45 data_preprocessing:413] 99 percentile: 16.70


100%|██████████| 1/1 [00:00<00:00, 13.16batch/s]


Query   : we bought four shirts and one mug from the nvidia gear store in santa clara
Combined: We bought four shirts and one mug from the Nvidia gear store in Santa Clara.

Query   : what can i do for you today
Combined: What can I do for you today?

Query   : how are you
Combined: How are you?

Query   : how is the weather in
Combined: How is the weather in?



Inference for large amounts of text can be performed by a script [examples/nlp/token_classification/punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py)

```
python punctuate_capitalize_infer.py \
    --input_manifest <PATH/TO/INPUT/MANIFEST> \
    --output_manifest <PATH/TO/OUTPUT/MANIFEST> \
    --pretrained_name punctuation_en_bert \
    --max_seq_length 64 \
    --margin 16 \
    --step 8
```

`<PATH/TO/INPUT/MANIFEST>` is a path to NeMo [ASR manifest](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/asr/datasets.html) which contains text in which you need to restore punctuation and capitalization. If manifest contains `'pred_text'` key, then `'pred_text'` elements will be processed. Otherwise, punctuation and capitalization will be restored in `'text'` elements.

`<PATH/TO/OUTPUT/MANIFEST>` is a path to NeMo ASR manifest into which result will be saved. The text with restored
punctuation and capitalization is saved into `'pred_text'` elements if `'pred_text'` key is present in
input manifest. Otherwise result will be saved into `'text'` elements.

Alternatively you can pass text for restoring punctuation and capitalization as plain text. See help for parameters `--input_text` and `--output_text` of the script
[punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py).

The script `examples/nlp/token_classification/punctuate_capitalize_infer.py` can restore punctuation and capitalization in a text of arbitrary length. Long sequences are split into segments
`--max_seq_length - 2` tokens each. Each segment starts and ends with `[CLS]` and `[SEP]`
tokens correspondingly. Every segment is offset to the previous one by `--step` tokens. For example, if
every character is a token, `--max_seq_length=5`, `--step=2`, then text `"hello"` will be split into
segments `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]`.

If segments overlap, then predicted probabilities for a token present in several segments are multiplied before selecting the best candidate.

Splitting leads to pour performance of a model near edges of segments. Use parameter `--margin` to discard `--margin`
probabilities predicted for `--margin` tokens near segment edges. For example, if every character is a token, `--max_seq_length=5`, `--step=1`, `--margin=1`, then text `"hello"` will be split into segments `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'e', 'l', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]`. Before calculating actual predictions, probabilities for tokens marked by asterisk are removed: `[['[CLS]', 'h', 'e', 'l'*, '[SEP]'*], ['[CLS]'*, 'e'*, 'l', 'l'*, '[SEP]'*], ['[CLS]'*, 'l'*, 'l', 'o', '[SEP]']]`.

You may use parameters `max_seq_length`, `step`, and `margin` in `add_punctuation_capitalization` method.

The text in the following example is from IWSLT 2019 test dataset.

In [ ]:
inference_results = pretrained_model.add_punctuation_capitalization(
    [
        "each of these songs represents a scene a movement in some cases a sonic revolution that completely altered the "
        "course of popular music they're all also calling cards almost for those cities songs totally linked with their "
        "city's identity and might be why you probably consider them to be music cities now the magical mythical thing "
        "the thing we kind of all love about stories like these is that those cities weren't doing anything in particular "
        "to make those moments happen there's no formula for capturing lightning in a bottle a formula didn't give us "
        "grunge music or introduce tupock to dr dray and there's definitely no blueprint for how to open your record "
        "business in a south memphis neighborhood that turns out is home to booker t jones william bell and albert king "
        "so this is just something that happens and right when the stars perfectly align great music just happens and "
        "in the meantime new york and nashville can churn out the hits that come through our radios define our "
        "generations and soundtrack our weddings and our funerals and everything in between i don't know about you but "
        "the very idea of that is just deadly boring to me there are musicians all around you making powerful important "
        "music and thanks to the internet and it's limitless possibilities for creators to create music and fans to "
        "discover that music those zyite guy songs don't have to be handed down to us from some conference room full of "
        "songwriters in a corporate high rise but also and more importantly we can't decide that it's just something that "
        "happens because music is about so much more than hits those big iconic moments that change everything it's more "
        "than just entertainment for so many of us music is truly a way to navigate life a means of self expression sure "
        "but it also helps us find ourrselfel worse and figure out who we are it connects us with other people as almost "
        "nothing else can across language barriers across social and cultural and economic divides music makes us smarter "
        "and healthier and happier music is necessary what if you lived in a city that believed that that said we're not "
        "waiting for that hit song to define us we're a music city because music is necessary by seeing music as "
        "necessary a city can build two things 1st an ecosystem to support the development of professional musicians "
        "and music business and 2nd a receptive and engaged audience to sustain them and those are the two critical "
        "elements of a music city a city whose leaders recognize the importance of music for our development as "
        "individuals our connection as a community and our viability as a vibrant place to live see smart cities music "
        "cities know that thriving night lifef a creative class culture is what attracts young talented people to cities "
        "it's what brings that lightning and no we can't predict the next egg that will hatch but we can create a city "
        "that acts like an incubator to do that 1st we got to know what we've got that means identifying and quantifying "
        "our assets we need to know them backward and forward from who and what and where they are to what their impact "
        "is on the economy let's count our recording studios and our record labels our historic landmarks and our "
        "hardcore punk clubs we should count monthly free jazz nights and weekly folk jams music schools artist "
        "development instrument shops every lay than every luther music museums open once a open year round and music "
        "festivals open just one weekend year now ideally through this process we'll create an actual asset map "
        "dropping a pin for each one allowing us to see exactly what we've got and where organic momentum is already "
        "happening because it's not enough to paint in broad strokes here when it comes to specific support for music "
        "locally and a broad understanding of a music brand nationally you've got to have the receipts next we'll need "
        "to identify our challenges now it's important to knowe that for the most part this won't be just the opposite "
        "of step one we won't gain a whole lot by simply thinking about what's missing from our map instead we need to "
        "approach this more holistically there are lots of music venues on our map awesome but are they struggling do "
        "we have a venue ladder which just means can an artist starting out at a coffee house open mike see a clear path "
        "for how they'll grow from that 25 seat room to a hundred seat room and so on or are we expecting them to go from "
        "a coffee house to a colossum maybe our challenges lie in city infrastructure public transportation affordable "
        "housing maybe like in london where the number of music venues went from 400 in 2010 to 100 in 2015 we need to "
        "think about protections against gentrification the mayor of london in december of last year actually added "
        "something called the agent of change principle to the city's comprehensive plan the name says it all if a real "
        "estate developer wants to build condos next to an existing music venue the developer is the agent of change they "
        "have to take the necessary steps for noise mitigation next and this is a very big one we need leadership and we "
        "need a strategy now we know there's a lot of magic in this mix a lot of right people right place right time and "
        "that will never stop being an important element of the way music is made the way some of the best most enduring "
        "music is made but there cannot be a leadership vacuum in 2018 surriving music cities don't often happen and "
        "don't have to happen accidentally we need elected officials who recognize the power of music and elevate the "
        "voices of creatives and they're ready to put a strategy in place in music cities from berlin to paris to bogata "
        "music advisory councils ensure that musicians have a seat at the table they're volunteer councils and they work "
        "directly with a designated advocate inside of city hall or even the chamber of commerce the strongest strategies "
        "will build music community supports like this one inward while also exporting music outward they go hand in hand "
        "when we look inward we create that place that musicians want to live and when we look outward we build "
        "opportunities for them to advance their career while also driving attention back to our city and leveraging music "
        "as a talent attraction tool and here's something else that will help with that we've got to figure out who we are "
        "now when i say austin you probably think live music capital and why because in 1991 leadership in austin saw "
        "something percolating with an existing asset and they chose to own it by recognizing that momentum naming it and "
        "claiming it they inevitably caused more live music venues to open existing spaces to add live music to their "
        "repertoire and they created a swell of civic buy in around the idea which meant that it wasn't just a slogan in "
        "some tourism pamphlet was something that locals really started to believe and take pride in now generally "
        "speaking what austin created is just an assets based narrative and when we think back to step one we know that "
        "every city will not tick every box many cities won't have recording studios like memphis or a songwriter and "
        "publishing scene like nashville and that's not a deal breaker we simply have to find the momentum happening in "
        "our city what are our unique assets in comparison to no other place so if all of that sounds like something "
        "you'd like to happen where you live here are three things you can do to move the needle 1st you can use your "
        "feet your ears and your dollars show up be that receptive and engaged audience that is so necessary for a music "
        "city to thrive pay a cover charge buy a record discover new music and please take your friends two you can use "
        "your voice buy in to the assets based narrative talk about and celebrate what your city has and three you can "
        "use your vote seek out leadership that doesn't just pay lip service to your city's music but recognizes its "
        "power and is prepared to put a strategy in place to elevate it grow it and build collaboration no there really "
        "is no telling what city could be defined by a certain scene or a certain song in the next decade but as much "
        "as we absolutely cannot predict that what we absolutely can predict is what happens when we treat music as "
        "necessary and we work to build a music city and that is a place where i want to live thank you"
    ],
    max_seq_length=128,
    step=8,
    margin=16,
    batch_size=32,
)
print(inference_results[0])

## Training Script

If you have NeMo installed locally, you can also train the model with [nlp/token_classification/punctuation_capitalization_train_evaluate.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/punctuation_capitalization_train_evaluate.py).

To run training script, use:

`python punctuation_capitalization_train_evaluate.py model.train_ds.ds_item=PATH_TO_TRAIN_DATA_DIR`

Set NUM_SAMPLES=-1 and consider including other datasets to improve the performance of the model.

# Finetuning model with your data

When we were training the model from scratch, the datasets were prepared for training during the model initialization. When we are using a pretrained Punctuation and Capitalization model, before training, we need to setup training and evaluation data.

In [ ]:
# let's reload our pretrained model
pretrained_model = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained('punctuation_en_distilbert')

# setup train and validation Pytorch DataLoaders
pretrained_model.update_config_after_restoring_from_checkpoint(
    train_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_train.txt',
        'labels_file': 'labels_train.txt',
        'tokens_in_batch': 1024,
    },
    validation_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_dev.txt',
        'labels_file': 'labels_dev.txt',
        'tokens_in_batch': 1024,
    },
)

# and now we can create a PyTorch Lightning trainer and call `fit` again
# for this tutorial we are setting fast_dev_run to True, and the trainer will run 1 training batch and 1 validation batch
# for actual model training, disable the flag
fast_dev_run = True
trainer = pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
pretrained_model.set_trainer(trainer)
pretrained_model.setup_training_data()
pretrained_model.setup_validation_data()
trainer.fit(pretrained_model)